In [1]:
# pip install jupyterthemes
# chesterish, grade3, gruvboxd, gruvboxl, monokai, oceans16, onedork, solarizedd, solarizedl
!jt -t grade3

In [2]:
%%javascript
// stops cursor blinking
var rate = 0;
IPython.notebook.get_cells().map(
    function(c) {  return c.code_mirror.options.cursorBlinkRate=rate;  }
);
// make sure new CodeMirror instance also use this setting
CodeMirror.defaults.cursorBlinkRate=rate;

<IPython.core.display.Javascript object>

In [3]:
%%javascript
IPython.notebook.events.off('checkpoint_created.Notebook');
IPython.notebook.events.off('notebook_saved.Notebook');

<IPython.core.display.Javascript object>

In [4]:
pip install scipy numpy pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Import libraries.
from pathlib import Path
import pandas as pd
import numpy as np
import codecs
import json
import time
import re
import os

In [6]:
# Lists the absolute file paths of files found in directory.
def absoluteFilePaths(directory):
    for dirpath,_,filenames in os.walk(directory):
        for f in filenames:
            yield os.path.abspath(os.path.join(dirpath, f))

In [7]:
# Reads into memory the json exports found in subdirectories 'REVIEW' and 'DOCUMENT'.
path_in = 'D:\\career\\previous_work\\rentgrata\\exports'
good_reads = [x[0] for x in os.walk(path_in) if 'DOCUMENT' in x[0] or 'REVIEW' in x[0]]
abs_reads = []
for gr in good_reads:
    abs_reads += [ar for ar in absoluteFilePaths(gr)]
exports = dict()
for ar in abs_reads:
    with open(ar, mode='r', encoding='utf-8') as f:
        exports[ar] = json.loads(f.read())
sample_path = list(exports.keys())[0]
exports[sample_path].keys() # Here are the keys to one such export.

dict_keys(['sentences', 'labelerInfo', 'labelSets', 'labels'])

In [8]:
# Extracts 'sentences' from exports.
filename_rgx = r'((prospect|resident)_[0-9]{8,}_[0-9]{8,})'
sentences = []
for abs_path, export_dct in exports.items():
    df = pd.DataFrame(export_dct['sentences'])
    df['abs_path'] = abs_path
    df['filename'] = re.search(filename_rgx, abs_path).group()
    df['review'] = 'REVIEW' in abs_path
    df.rename(columns={'id': 'sentence_number'}, inplace=True) 
    df.set_index(['abs_path', 'sentence_number'], inplace=True)
    sentences.append(df.copy())
sentences = pd.concat(sentences)
del sentences['labels']
sentences[['content', 'tokens', 'review']].head()

content  \
abs_path                                           sentence_number                                                
D:\career\previous_work\rentgrata\exports\prosp... 0                              I m relocating from Minnesota   
                                                   1                                            Have a nice day   
                                                   2                                          Hey , this is Jim   
                                                   3                I am planning to rent a studio in this area   
                                                   4                                    Do you like living here   

                                                                                                               tokens  \
abs_path                                           sentence_number                                                      
D:\career\previous_work\rentgrata\exports\prosp... 0                              [I, m, relocating, from, Minnesota]   
                                                   1                                             [Have, a, nice, day]   
                                                   2                                          [Hey, ,, this, is, Jim]   
                                                   3                [I, am, planning, to, rent, a, studio, in, thi...   
                                                   4                                    [Do, you, like, living, here]   

                                                                    review  
abs_path                                           sentence_number          
D:\career\previous_work\rentgrata\exports\prosp... 0                  True  
                                                   1                  True  
                                                   2                  True  
                                                   3                  True  
                                                   4                  True

In [9]:
# Extracts 'labelSets' from exports to map unfriendly 'labelSetItemId' to friendly 'label'.
labelSet = 1 # counting starts at 1
mappers = {}
for abs_path, export_dict in exports.items():
    df = pd.DataFrame(export_dict['labelSets'][labelSet-1]['labelItems'])
    df.set_index('id', inplace=True)
    mappers[abs_path] = df['labelName'].copy()
mappers[sample_path].head() # Here is one such mapper.

id
bzwgd0hD2G1rDlWjU6FhF    Proprietary_Entity_1
c2kNhq3OT8k5f3DTkI1qY    Proprietary_Entity_2
uk0btwAdF-pKP9PsLOXTx    Proprietary_Entity_3
Aq_Evg6XN1CkZM4D8Q3Mp    Proprietary_Entity_4
TGfPD1P8veFJHK8d0PskZ    Proprietary_Entity_5
Name: labelName, dtype: object

In [10]:
# Extracts labels from exports.
labels = []
for abs_path, export_dct in exports.items():
    if len(export_dct['labels']) > 0:
        df = pd.DataFrame(export_dct['labels'])
    else:
        continue
    df['abs_path'] = abs_path
    df['label'] = df['labelSetItemId'].map(mappers[abs_path])
    mask = df['startCellLine'] == df['endCellLine']
    df = df.loc[mask, :].copy()
    df.rename(columns={'startCellLine': 'sentence_number'}, inplace=True)
    df.set_index(['abs_path', 'sentence_number'], inplace=True)
    labels.append(df.copy())
labels = pd.concat(labels)
labels['label_type'] = labels['label'].isin(['Negative', 'Neutral', 'Positive'])
labels['label_type'].replace({True: 'Sentiment', False: 'Entity'}, inplace=True)
labels[['label_type', 'label']].head()

label_type  \
abs_path                                           sentence_number              
D:\career\previous_work\rentgrata\exports\resid... 1                Sentiment   
                                                   1                   Entity   
                                                   1                   Entity   
                                                   1                Sentiment   
                                                   2                   Entity   

                                                                                   label  
abs_path                                           sentence_number                        
D:\career\previous_work\rentgrata\exports\resid... 1                             Neutral  
                                                   1                Proprietary_Entity_6  
                                                   1                Proprietary_Entity_7  
                                                   1                             Neutral  
                                                   2                Proprietary_Entity_8

In [11]:
# Joins sentences with labels.
df = sentences.join(labels)
df.dropna(subset=['startTokenIndex', 'endTokenIndex', 'label'], inplace=True)
df['startTokenIndex'] = df['startTokenIndex'].astype(int)
df['endTokenIndex'] = df['endTokenIndex'].astype(int)
df[['content', 'label_type', 'label', 'review']].head()
mask = df['tokens'].apply(len) > 0
df = df.loc[mask, :].copy()
df['user'] = df['filename'].apply(lambda x: x[:8])
df[['startTokenIndex', 'endTokenIndex', 'label']].head()

startTokenIndex  \
abs_path                                           sentence_number                    
D:\career\previous_work\rentgrata\exports\resid... 1                              0   
                                                   1                              0   
                                                   1                              9   
                                                   1                              9   
                                                   2                              1   

                                                                    endTokenIndex  \
abs_path                                           sentence_number                  
D:\career\previous_work\rentgrata\exports\resid... 1                            7   
                                                   1                            7   
                                                   1                           12   
                                                   1                           12   
                                                   2                            8   

                                                                                   label  
abs_path                                           sentence_number                        
D:\career\previous_work\rentgrata\exports\resid... 1                             Neutral  
                                                   1                Proprietary_Entity_6  
                                                   1                Proprietary_Entity_7  
                                                   1                             Neutral  
                                                   2                Proprietary_Entity_8

In [12]:
# Extracts phrase from sentence; favors reviewer's label over labeler's label.
sortby = ['filename', 'sentence_number', 'startCharIndex', 'endCharIndex', 'label_type', 'review']
dedup  = ['filename', 'sentence_number', 'startCharIndex', 'endCharIndex', 'label_type']
df['phrase'] = df.apply(lambda x: x['tokens'][x['startTokenIndex']:x['endTokenIndex']+1], axis=1)
df['phrase'] = df['phrase'].apply(lambda x: ' '.join(x))
df['startCharIndex'] = df.apply(lambda x: x['content'].index(x['phrase']), axis=1)
df['startCharIndex'] = df['startCharIndex'].astype(int)
df['endCharIndex'] = df['startCharIndex'] + df['phrase'].apply(len)
df['endCharIndex'] = df['endCharIndex'].astype(int)
df.reset_index(drop=False, inplace=True)
df.sort_values(by=sortby, ascending=True, inplace=True)
df.drop_duplicates(subset=dedup, keep='last', inplace=True)
df[sortby].head()

,filename,sentence_number,startCharIndex,endCharIndex,label_type,review
113156,resident_00001000_00001199,0,3,28,Entity,True
113157,resident_00001000_00001199,0,3,28,Sentiment,True
38926,resident_00001000_00001199,0,33,57,Entity,False
38927,resident_00001000_00001199,0,33,57,Sentiment,False
113158,resident_00001000_00001199,1,7,32,Entity,True


In [13]:
# Separates sentiment labels from entity labels.
sentiment_mapper = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
mask = df['label_type'] == 'Sentiment'
snt = df.loc[mask, :].copy()
snt.set_index(['filename', 'sentence_number', 'startCharIndex', 'endCharIndex'], inplace=True)
snt['label'] = snt['label'].map(sentiment_mapper)
mask = df['label_type'] == 'Entity'
df = df.loc[mask, :].copy()

# Reformats entity labels for Google.
df['offsets'] = df.apply(lambda x: dict([('startOffset', str(x['startCharIndex'])),
                                          ('endOffset',  str(x['endCharIndex']))]), axis=1)
df['textSegment'] = df['offsets'].apply(lambda x: dict([('textSegment', x)]))
df['textExtraction'] = df['textSegment'].apply(lambda x: dict([('textExtraction', x)]))
df['displayName'] = df['label'].apply(lambda x: dict([('displayName', x)]))
df['displayName'] = df.apply(lambda x: {**x['displayName'], **x['textExtraction']}, axis=1)

# Groups entity labels by sentence.
gb = df.groupby(['filename', 'sentence_number'])
df2 = gb.apply(lambda x: x['displayName'].tolist()).to_frame('annotations')
df2['user'] = gb['user'].first()
df2['annotations'] = df2['annotations'].apply(lambda x: dict([('annotations', x)]))
df2['textSnippet'] = gb['content'].first()
df2['textSnippet'] = df2.apply(lambda x:
                               dict([('textSnippet',
                                      dict([('content', x['textSnippet'])]))]),
                               axis=1)
df2['jsonl'] = df2.apply(lambda x: {**x['textSnippet'], **x['annotations']}, axis=1)
sample_jsonl = df2['jsonl'].values.tolist()[0]
print(sample_jsonl['textSnippet'])
print(sample_jsonl['annotations'][0]) # Here is one such reformatted label and phrase from a sentence.

{'content': 'My roommate uses that closet but does not use other closets'}
{'displayName': 'Proprietary_Entity_9', 'textExtraction': {'textSegment': {'startOffset': '3', 'endOffset': '28'}}}


In [14]:
# Shows label counts.
print('{:,} entity labels'.format(df2.shape[0]))
print('{:,} sentiment labels'.format(snt.shape[0]))

40,219 entity labels
55,555 sentiment labels
